In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 100]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))

[ 1  2  3  4  5  6  7  8  9 10] [955020     14 240816     19   4000 467856      3   1083     52   2408]
(array([ 1,  3,  5,  6,  8, 10]), array([955020, 240816,   4000, 467856,   1083,   2408]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '3' encode 1
defense '5' encode 2
defense '6' encode 3
defense '8' encode 4
defense '10' encode 5


In [5]:
# Collect 1% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_1, _ , y_1 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, stratify=y_label_0_1, random_state=42)

In [6]:
X_1.shape

(16712, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt1 = DecisionTreeClassifier()
# dt1.fit(X_1, y_1)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 6] [     4 238749]
(4, 42) (4,)
Save 1 to baseline_Def1.npy
(238749, 42) (238749,)
Save 6 to baseline_Def6.npy
Execution Time: 0.259468 seconds
(716259,)
[ 1  3  5  6 10] [248620 220279   8074 238785    501]
(248620, 42) (248620,)
Save 1 to BIM_Def1.npy
(220279, 42) (220279,)
Save 3 to BIM_Def3.npy
(8074, 42) (8074,)
Save 5 to BIM_Def5.npy
(238785, 42) (238785,)
Save 6 to BIM_Def6.npy
(501, 42) (501,)
Save 10 to BIM_Def10.npy
Execution Time: 0.428841 seconds
(716259,)
[ 1  3  5  6  8 10] [406872  37977  14704 251927   2021   2758]
(406872, 42) (406872,)
Save 1 to FGSM_Def1.npy
(37977, 42) (37977,)
Save 3 to FGSM_Def3.npy
(14704, 42) (14704,)
Save 5 to FGSM_Def5.npy
(251927, 42) (251927,)
Save 6 to FGSM_Def6.npy
(2021, 42) (2021,)
Save 8 to FGSM_Def8.npy
(2758, 42) (2758,)
Save 10 to FGSM_Def10.npy
Execution Time: 0.395554 seconds
(716259,)
[ 1  3  5  6 10] [248620 220279   8074 238785    501]
(248620, 42) (248620,)
Save 1 to PGD_Def1.npy
(220279, 42) (220279,)
Save 3 to PG

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 6] [     3 238750]
(3, 42) (3,)
Save 1 to baseline_Def1.npy
(238750, 42) (238750,)
Save 6 to baseline_Def6.npy
Execution Time: 0.651935 seconds
(716259,)
[ 1  3  5  6 10] [447160  29260    924 238910      5]
(447160, 42) (447160,)
Save 1 to BIM_Def1.npy
(29260, 42) (29260,)
Save 3 to BIM_Def3.npy
(924, 42) (924,)
Save 5 to BIM_Def5.npy
(238910, 42) (238910,)
Save 6 to BIM_Def6.npy
(5, 42) (5,)
Save 10 to BIM_Def10.npy
Execution Time: 4.175219 seconds
(716259,)
[ 1  3  5  6 10] [225824   5734   8725 472276   3700]
(225824, 42) (225824,)
Save 1 to FGSM_Def1.npy
(5734, 42) (5734,)
Save 3 to FGSM_Def3.npy
(8725, 42) (8725,)
Save 5 to FGSM_Def5.npy
(472276, 42) (472276,)
Save 6 to FGSM_Def6.npy
(3700, 42) (3700,)
Save 10 to FGSM_Def10.npy
Execution Time: 3.913660 seconds
(716259,)
[ 1  3  5  6 10] [447160  29260    924 238910      5]
(447160, 42) (447160,)
Save 1 to PGD_Def1.npy
(29260, 42) (29260,)
Save 3 to PGD_Def3.npy
(924, 42) (924,)
Save 5 to PGD_Def5.npy
(238910, 42) (23

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt1.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude2Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude2Attack/WUSTL_Input1/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Exclude2Attack/WUSTL_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_Exclude2Attack.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,4,0,0,0,0,238749,0,0,0,0,0,238753
1,XGB1,248620,0,220279,0,8074,238785,0,0,0,501,0,716259
2,XGB1,406872,0,37977,0,14704,251927,0,2021,0,2758,0,716259
3,XGB1,248620,0,220279,0,8074,238785,0,0,0,501,0,716259
4,XGB1,141,0,5473,0,12592,687340,0,4483,0,6230,0,716259
5,XGB1,249118,0,190091,0,2984,271164,0,2033,0,869,0,716259
6,XGB1,12,0,0,0,0,716247,0,0,0,0,0,716259
7,XGB1,198290,0,3844,0,1117,512975,0,0,0,33,0,716259
8,XGB1,1182,0,474591,0,1418,239055,0,1,0,12,0,716259
9,XGB1,434970,0,33097,0,3745,242803,0,1634,0,10,0,716259
